### KEN 3140: Lab 3 (RDFS reasoning)
**Author:** Kody Moodley  
**Date:** 2021-09-01  
**License:** [https://creativecommons.org/licenses/by/4.0](https://creativecommons.org/licenses/by/4.0)

In this lab we are going to execute inferences made on RDF triples that make use of RDFS language features (using another Java-based Semantic Web library called [Apache Jena](https://jena.apache.org/)).

#### Learning objectives:
1. Be able to identify which knowledge representation features are not included in [Resource Description Framework (RDF)](https://www.w3.org/TR/rdf11-concepts/) and are new to its extension: [RDFS](https://www.w3.org/TR/rdf-schema/)
2. Be able to identify and add suitable [domain](https://www.w3.org/TR/rdf-schema/#ch_domain), [range](https://www.w3.org/TR/rdf-schema/#ch_range) and [subclass](https://www.w3.org/TR/rdf-schema/#ch_subclassof) relations to an RDF graph to capture more detailed meaning for the entities in the graph
3. Be able to apply RDFS reasoning services to an RDF graph to compute implicit inferences about the graph which are not explicitly indicated

### Setup Apache Jena
We will use another RDF Java library in this lab called [Apache Jena](https://jena.apache.org/) because the reasoning features are more stable and easier to use than [RDF4J](https://rdf4j.org/documentation/programming/spin/). Follow these instructions to install the library:

1. Extract the ``apache-jena-3.16.0.zip`` archive to the same directory as the notebook.
2. Run the command ``%jars apache-jena-3.16.0/lib/*.jar`` in the next cell.

In [2]:
%jars apache-jena-3.16.0/lib/*.jar

Import the libraries we will use from Jena:

In [3]:
import org.apache.jena.riot.RDFDataMgr;
import org.apache.jena.rdf.model.*;
import org.apache.jena.util.PrintUtil;
import org.apache.jena.vocabulary.RDF;

### Lab Tasks
You are provided with a pre-prepared RDF document in your lab materials called ``Lab3_RDFS_graph.ttl``. In the text editor of your choice or in the Jupyter Lab environment, open and study the RDF graph located in this file. The graph is represented in Turtle syntax. Notice that in Turtle syntax comments are preceded by the ``#`` character so everything after this character on a particular line is ignored by RDF parsers like Jena. You will also notice there are two sections marked ``Schema section`` and ``Data section`` respectively. The triples under the section titled ``Schema section`` are currently all commented out.

#### A. Import Lab 3 example graph, build RDFS model and print triples
Run the code below to import the RDFS graph in file ``Lab3_RDFS_graph.ttl`` into memory and to print some triples from it.

In [4]:
// Function to print out triples in our graph where the subject, predicate and object match "s", "p", "o" 
public void printStatements(Model m, Resource s, Property p, Resource o) { 
    for (StmtIterator i = m.listStatements(s,p,o); i.hasNext(); ){ 
        Statement stmt = i.nextStatement(); 
        System.out.println(PrintUtil.print(stmt));
    } 
}

// Load and create Jena RDF model from file
Model schema = RDFDataMgr.loadModel("Lab3_RDFS_graph.ttl");
InfModel infmodel = ModelFactory.createRDFSModel(schema);

// Create references to resources in the graph (kevin and angela)
Resource kevin = infmodel.getResource("http://example.com/kevin");
Resource angela = infmodel.getResource("http://example.com/angela");

// Print out relevant triples
System.out.println("Triples where Kevin is the subject:");
printStatements(infmodel, kevin, null, null);
System.out.println("\nTriples where Angela is the subject:");
printStatements(infmodel, angela, null, null);

Triples where Kevin is the subject:
(<http://example.com/kevin> <http://example.com/hasAge> '17.5'^^http://www.w3.org/2001/XMLSchema#integer)
(<http://example.com/kevin> rdf:type <http://example.com/Teenager>)
(<http://example.com/kevin> <http://example.com/hasMother> <http://example.com/angela>)
(<http://example.com/kevin> <http://example.com/hasParent> <http://example.com/angela>)

Triples where Angela is the subject:
(<http://example.com/angela> <http://example.com/isMotherOf> <http://example.com/kevin>)


**Take careful note of the triples printed above.**

#### B. Tasks for this lab
Carefully examine the triples in the file ``Lab3_RDFS_graph.ttl`` provided with your lab materials. You will notice that each triple in the ``Schema`` section of this file has a lab task number associated with it. For each task below, you must identify which triple(s) in the ``Schema`` section are associated with that task and then uncomment it to see its effect on the RDFS inferences obtained.

To uncomment a triple, remove only the **first** ``#`` character on the line for that triple. **Do not remove the second ``#`` character on the line**. After each modification, save ``Lab3_RDFS_graph.ttl`` and then re-run the code in **Cell 3A. Build RDFS model and print triples**

**Task 1.** Uncomment the triples associated with ``Lab 3 - Task 1`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?  

**Task 2.** Uncomment the triples associated with ``Lab 3 - Task 2`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?  

**Task 3.** Uncomment the triples associated with ``Lab 3 - Task 3`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?

### FAQs

##### 1. Are there tools to visualise an RDF graph?

Yes. There are many proprietary and free tools for RDF graph management that have visualisation features e.g. [GraphDB](https://graphdb.ontotext.com/) and [Allegrograph](https://allegrograph.com/). However, real-world RDF graphs tend to be extremely large (with millions and sometimes billions of entities). Therefore, it is often not practical to visualise an entire RDF graph but rather smaller subsets or specialised "views" of this graph which is what these tools focus on.

For **small** graphs, a very quick to use tool is called RDF-grapher and can be located on this page: [http://www.ldf.fi/service/rdf-grapher](http://www.ldf.fi/service/rdf-grapher). You can copy and paste your Turtle representation into the textbox and click on the ``Visualize`` button. This service is mentioned here just out of interest. You will not be examined about this tool in the course. If it does not work, that's okay - it is not required for the course that it does. We are not going to provide support on how to make it work. [RDF-grapher](http://www.ldf.fi/service/rdf-grapher) does not produce the prettiest visualisation but is generally reliable for producing functional visualisations for small RDF graphs.

##### 2. Is RDFS reasoning rule-based?

Technically, no. However, in practice, the specific software implementations of RDFS reasoning may employ rule-based algorithms. However, theoretically, RDF and RDFS have a [formal semantics](https://www.w3.org/TR/rdf11-mt/) which means that what we can infer from a set of RDFS statements is not necessarily a matter of applying an "if-then" rule. RDFS reasoning is, however, [deductive](https://en.wikipedia.org/wiki/Deductive_reasoning) as opposed to [inductive](https://en.wikipedia.org/wiki/Inductive_reasoning).

##### 3. Is it possible to use inductive reasoning e.g. Machine Learning and Deep Learning on RDF graphs as well?
Yes! We teach some of these techniques in our Masters course at DKE on [Building and Mining Knowledge Graphs](https://www.maastrichtuniversity.nl/meta/371276/building-and-mining-knowledge-graphs) which builds on many of the technologies we cover in this course such as RDF and [SPARQL](https://www.w3.org/TR/rdf-sparql-query/).